In [61]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [62]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN','NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [63]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [64]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [65]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 500
application_types_to_replace = []
for app in application_df['APPLICATION_TYPE'].value_counts().index:
  if application_df['APPLICATION_TYPE'].value_counts()[app] < cutoff:
    application_types_to_replace.append(app)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [66]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [67]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1000
classifications_to_replace = []
for cls in application_df["CLASSIFICATION"].value_counts().index:
  if application_df["CLASSIFICATION"].value_counts()[cls] < cutoff:
    classifications_to_replace.append(cls)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [68]:
# Look at ASK_AMT value counts for binning
application_df["ASK_AMT"].value_counts().sort_index()

5000          25398
5001              1
5006              2
5016              1
5037              1
              ...  
3391919220        1
4653011914        1
5591584994        1
8556638692        1
8597806340        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [69]:
# Choose cutoff values and create bins accordingly
cutoffs = [1000000000,100000000,1000000,100000,50000,10000]
for amt in application_df["ASK_AMT"].value_counts().index:
  if amt > cutoffs[0]:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,">1 Billion")
  elif amt > cutoffs[1]:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,"100 Million-1 Billion")
  elif amt > cutoffs[2]:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,"1 Million-100 Million")
  elif amt > cutoffs[3]:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,"100 Thousand-1 Million")
  elif amt > cutoffs[4]:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,"50 Thousand-100 Thousand")
  elif amt > cutoffs[5]:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,"10 Thousand-50 Thousand")
  else:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(amt,"<10 Thousand")

# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

<10 Thousand                25947
100 Thousand-1 Million       2954
10 Thousand-50 Thousand      2398
1 Million-100 Million        1476
50 Thousand-100 Thousand     1423
100 Million-1 Billion          83
>1 Billion                     18
Name: ASK_AMT, dtype: int64

In [70]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df)
application_dummies_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_1 Million-100 Million,ASK_AMT_10 Thousand-50 Thousand,ASK_AMT_100 Million-1 Billion,ASK_AMT_100 Thousand-1 Million,ASK_AMT_50 Thousand-100 Thousand,ASK_AMT_<10 Thousand,ASK_AMT_>1 Billion
0,1,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
1,1,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
3,1,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [74]:
# Split our preprocessed data into our features and target arrays
y = application_dummies_df["IS_SUCCESSFUL"]
X = application_dummies_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)

In [75]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [79]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(4, activation="relu", name="layer1", input_dim=49))

# Second hidden layer
nn.add(tf.keras.layers.Dense(8, activation="relu", name="layer2"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(8, activation="relu", name="layer3"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="tanh"))

# Check the structure of the model
nn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 4)                 200       
                                                                 
 layer2 (Dense)              (None, 8)                 40        
                                                                 
 layer3 (Dense)              (None, 8)                 72        
                                                                 
 dense_7 (Dense)             (None, 1)                 9         
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [80]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [81]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 2ms/step - loss: 1.4687 - accuracy: 0.4572
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6616 - accuracy: 0.6208
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6405 - accuracy: 0.6478
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.6023 - accuracy: 0.7042
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5837 - accuracy: 0.7115
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5759 - accuracy: 0.7163
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5713 - accuracy: 0.7200
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5659 - accuracy: 0.7224
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5672 - accuracy: 0.7240
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5639 - accuracy: 0.7261

In [82]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5592 - accuracy: 0.7265 - 708ms/epoch - 3ms/step
Loss: 0.5591771602630615, Accuracy: 0.7265306115150452


In [84]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(8, activation="relu", name="layer1", input_dim=49))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(16, activation="relu", name="layer2"))

# Third hidden layer
nn2.add(tf.keras.layers.Dense(16, activation="relu", name="layer3"))

# Fourth hidden layer
nn2.add(tf.keras.layers.Dense(16, activation="relu", name="layer4"))


# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 8)                 400       
                                                                 
 layer2 (Dense)              (None, 16)                144       
                                                                 
 layer3 (Dense)              (None, 16)                272       
                                                                 
 layer4 (Dense)              (None, 16)                272       
                                                                 
 dense_9 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,105
Trainable params: 1,105
Non-trainable params: 0
_________________________________________________________________


In [85]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [86]:
# Train the model
fit_model2 = nn2.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.6042 - accuracy: 0.6906
Epoch 2/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5627 - accuracy: 0.7262
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5566 - accuracy: 0.7286
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5541 - accuracy: 0.7298
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7292
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7315
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5505 - accuracy: 0.7321
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5495 - accuracy: 0.7312
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5486 - accuracy: 0.7316
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7334

In [87]:
# Evaluate the model using the test data
model2_loss, model2_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model2_loss}, Accuracy: {model2_accuracy}")

268/268 - 0s - loss: 0.5556 - accuracy: 0.7283 - 474ms/epoch - 2ms/step
Loss: 0.5555945038795471, Accuracy: 0.7282798886299133


In [91]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(2, activation="relu", name="layer1", input_dim=49))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(4, activation="relu", name="layer2"))


# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 2)                 100       
                                                                 
 layer2 (Dense)              (None, 4)                 12        
                                                                 
 dense_11 (Dense)            (None, 1)                 5         
                                                                 
Total params: 117
Trainable params: 117
Non-trainable params: 0
_________________________________________________________________


In [92]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [93]:
# Train the model
fit_model3 = nn3.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 5s 5ms/step - loss: 0.6427 - accuracy: 0.6714
Epoch 2/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5799 - accuracy: 0.7214
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5687 - accuracy: 0.7274
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5663 - accuracy: 0.7278
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5647 - accuracy: 0.7271
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5640 - accuracy: 0.7275
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5636 - accuracy: 0.7275
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5633 - accuracy: 0.7280
Epoch 9/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5629 - accuracy: 0.7287
Epoch 10/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5628 - accuracy: 0.7288

In [95]:
# Evaluate the model using the test data
model3_loss, model3_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model3_loss}, Accuracy: {model3_accuracy}")

268/268 - 0s - loss: 0.5640 - accuracy: 0.7269 - 351ms/epoch - 1ms/step
Loss: 0.5639742612838745, Accuracy: 0.7268804907798767


In [96]:
# Export our model to HDF5 file
nn2.save_weights('AlphabetSoupCharity_Optimization.h5')